In [1]:
"""
REDO THIS WITHOUT KEEPING TRACK OF EDGES

Idea: among removed triangles, pair up faces that both apear, left with faces that don't - the boundary, from which we construct new triangles

have two lists, faces left to check, and faces to check against (these will be all 3 anticlockwise versions of each face)
keep track of the batch you came from, and the index against which you are currently checking
increase index by one each time until either: find a match, or: no longer checking against same batch
at which point we remove FROM THE FIRST LIST
repeat until all removed
when find a match, mark it in second list
removed all marked faces
somehow find number remaining in each batch, and make sure to copy that many 'new points' into a long list
construct new triangles from the above information
"""



import math
import random
import numpy as np
import torch


device = "cuda:0"
floattype = torch.float


class environment:    
    def reset(self, npoints, batchsize, nsamples=1, corner_points = None, initial_triangulation = None):
        """
        corner_points, etc., shoudn't include a batch dimension
        """
        if corner_points == None:
            ncornerpoints = 4
        else:
            ncornerpoints = corner_points.size(0)
        if npoints <= ncornerpoints:
            print("Error: not enough points for valid problem instance")
            return
        self.batchsize = (
            batchsize * nsamples
        )  # so that I don't have to rewrite all this code, we store these two dimensions together
        self.nsamples = nsamples
        self.npoints = npoints
        self.points = (
            torch.rand([batchsize, npoints - ncornerpoints, 3], dtype = floattype, device=device)
            .unsqueeze(1)
            .expand(-1, nsamples, -1, -1)
            .reshape(self.batchsize, npoints - ncornerpoints, 3)
        )
        if corner_points == None:
            self.corner_points = torch.tensor(
                [[0, 0, 0], [3, 0, 0], [0, 3, 0], [0, 0, 3]], dtype = floattype, device=device
            )
        else:
            self.corner_points = corner_points
        self.points = torch.cat(
            [
                self.corner_points.unsqueeze(0).expand(self.batchsize, -1, -1),
                self.points,
            ],
            dim=-2,
        )  # [batchsize * nsamples, npoints, 3]
        self.points_mask = torch.cat(
            [
                torch.ones([self.batchsize, ncornerpoints], dtype=torch.bool, device=device),
                torch.zeros(
                    [self.batchsize, npoints - ncornerpoints], dtype=torch.bool, device=device
                ),
            ],
            dim=1,
        )
        self.points_sequence = torch.empty(
            [self.batchsize, 0], dtype=torch.long, device=device
        )

        """
        points are now triples
        triangles are now quadruples
        edges are now still just indices, but there are four of them per 'triangle', and they correspond to triples of points, not pairs
        we use  0,2,1  0,3,2  0,1,3  1,2,3  as the order of the four 'edges'/faces
        opposite face is always ordered such that the last two indices are swapped
        faces are always read ANTICLOCKWISE
        
        first three points of tetrahedron MUST be read clockwise (from the outside) to get correct sign on incircle test
        
        new point will be inserted in zeroth position, so if corresponding face of REMOVED tetrahedron is [x,y,z] (being read anticlockwise from outside in) new tetrahedron is [p, x, y, z]
        """
        
        """
        number of tetrahedra is not the same for each batch (in 3D), so store as a big list, and remember batch index that it comes from
        """
        if corner_points == None:
            initial_triangulation = torch.tensor([[0, 1, 2, 3]], dtype=torch.long, device=device)
        
        self.partial_delaunay_triangles = initial_triangulation.unsqueeze(0).expand(self.batchsize, -1, -1).reshape(-1, 4)
        self.batch_index = torch.arange(self.batchsize, dtype = torch.long, device = device).unsqueeze(1).expand(-1, initial_triangulation.size(0)).reshape(-1)
        
        self.batch_triangles = self.partial_delaunay_triangles.size(0) #[0]
        self.ntriangles = torch.full([self.batchsize], initial_triangulation.size(0), dtype = torch.long, device = device) #[self.batchsize]
        
        self.cost = torch.zeros([self.batchsize], dtype = floattype, device=device)

        self.logprob = torch.zeros([self.batchsize], dtype = floattype, device=device, requires_grad=True)

    def update(self, point_index):  # point_index is [batchsize]
        
        assert point_index.size(0) == self.batchsize
        assert str(point_index.device) == device
        assert self.points_mask.gather(1, point_index.unsqueeze(1)).sum() == 0
        
        triangles_coordinates = self.points[self.batch_index.unsqueeze(1), self.partial_delaunay_triangles] # [batch_triangles, 4, 3]
        assert list(triangles_coordinates.size()) == [self.batch_triangles, 4, 3]
        
        newpoint = self.points[self.batch_index, point_index[self.batch_index]] # [batch_triangles, 3]
        assert list(newpoint.size()) == [self.batch_triangles, 3]

        incircle_matrix = torch.cat(
            [
                newpoint.unsqueeze(1),
                triangles_coordinates,
            ],
            dim=-2,
        )  # [batch_triangles, 5, 3]
        incircle_matrix = torch.cat(
            [
                (incircle_matrix * incircle_matrix).sum(-1, keepdim=True),
                incircle_matrix,
                torch.ones([self.batch_triangles, 5, 1], dtype = floattype, device=device),
            ],
            dim=-1,
        )  # [batch_triangles, 5, 5]
        assert incircle_matrix.dtype == floattype
        assert str(incircle_matrix.device) == device
        
        incircle_test = (
            incircle_matrix.det() > 0
        )  # [batch_triangles], is True if inside incircle
        assert list(incircle_test.size()) == [self.batch_triangles]
        
        conflicts = incircle_test.sum()
        
        conflicting_triangles = self.partial_delaunay_triangles[incircle_test] # [conflicts, 4]
        assert list(conflicting_triangles.size()) == [conflicts, 4]
        
        conflicting_edges_index0 = torch.empty_like(conflicting_triangles)
        indices = torch.LongTensor([0, 0, 0, 1])
        conflicting_edges_index0 = conflicting_triangles[:, indices] # [conflicts, 4]
        
        conflicting_edges_index1 = torch.empty_like(conflicting_triangles)
        indices = torch.LongTensor([2, 3, 1, 2])
        conflicting_edges_index1 = conflicting_triangles[:, indices] # [conflicts, 4]
        
        conflicting_edges_index2 = torch.empty_like(conflicting_triangles)
        indices = torch.LongTensor([1, 2, 3, 3])
        conflicting_edges_index2 = conflicting_triangles[:, indices] # [conflicts, 4]
        
        conflicting_edges = torch.cat([conflicting_edges_index0.view(-1).unsqueeze(-1), conflicting_edges_index1.view(-1).unsqueeze(-1), conflicting_edges_index2.view(-1).unsqueeze(-1)], dim = -1).reshape(-1, 3) # [conflicts * 4, 3]
        assert list(conflicting_edges.size()) == [conflicts * 4, 3]
        
        conflicting_edges_copy = conflicting_edges.clone() # [conflicts * 4, 3], for later use to construct the new triangles
        
        edge_batch_index = self.batch_index[incircle_test].unsqueeze(1).expand(-1, 4).reshape(-1) # [conflicts * 4]
        assert list(edge_batch_index.size()) == [conflicts * 4]
        assert str(edge_batch_index.device) == device
        
        indices = torch.LongTensor([0, 2, 1])
        comparison_edges = conflicting_edges[:, indices] # [conflicts * 4, 3]
        assert list(comparison_edges.size()) == [conflicts * 4, 3]
        
        unravel_nomatch_mask = torch.ones([conflicts * 4], dtype = torch.bool, device = device) # [conflicts * 4]
        
        edge_batch_index_extended = torch.cat([edge_batch_index, torch.tensor([-1], dtype = torch.long, device = device)])
        
        starting_index = torch.arange(conflicts * 4, dtype = torch.long, device = device) # [conflicts * 4]
        current_comparison_index = starting_index.clone() # [conflicts * 4]
        batch_index_todo = edge_batch_index.clone()
        todo_mask = torch.ones([conflicts * 4], dtype = torch.bool, device = device) # [conflicts * 4]
        while todo_mask.sum():
            
            if todo_mask.sum() < todo_mask.size(0) // 2:
                conflicting_edges = conflicting_edges[todo_mask]
                current_comparison_index = current_comparison_index[todo_mask]
                starting_index = starting_index[todo_mask]
                batch_index_todo = batch_index_todo[todo_mask]
                todo_mask = torch.ones([todo_mask.sum()], dtype = torch.bool, device = device)
            
            assert conflicting_edges.size(0) == todo_mask.size(0)
            assert current_comparison_index.size() == todo_mask.size()
            assert batch_index_todo.size() == todo_mask.size()
            
            nomatch_mask = (conflicting_edges != comparison_edges[current_comparison_index]).sum(-1).bool()#.logical_or(batch_index_todo != edge_batch_index_extended[current_comparison_index])
            
            match_mask = nomatch_mask.logical_not().logical_and(todo_mask)
            match_index0 = starting_index[match_mask]
            match_index1 = current_comparison_index[match_mask]
            assert unravel_nomatch_mask[match_index0].logical_not().sum().logical_not()
            assert unravel_nomatch_mask[match_index1].logical_not().sum().logical_not()
            unravel_nomatch_mask[match_index0] = False
            unravel_nomatch_mask[match_index1] = False
            
            todo_mask = todo_mask.logical_and(nomatch_mask).logical_and(batch_index_todo == edge_batch_index_extended[current_comparison_index + 1])
            
            current_comparison_index = (current_comparison_index + 1) * todo_mask
        
        batch_newtriangles = unravel_nomatch_mask.sum()
        
        nomatch_edges = conflicting_edges_copy[unravel_nomatch_mask] # [batch_newtriangles, 3], already in correct order to insert into 1,2,3 (since already anticlockwise from outside in)
        assert list(nomatch_edges.size()) == [batch_newtriangles, 3]
        nomatch_batch_index = edge_batch_index[unravel_nomatch_mask] # [batch_newtriangles]
        
        nomatch_newpoint = point_index[nomatch_batch_index] # [batch_newtriangles]
        
        newtriangles = torch.cat([nomatch_newpoint.unsqueeze(1), nomatch_edges], dim = -1) # [batch_newtriangles, 4]
        
        
        nremoved_triangles = torch.zeros([self.batchsize], dtype = torch.long, device = device)
        nnew_triangles = torch.zeros([self.batchsize], dtype = torch.long, device = device)
        
        indices = self.batch_index[incircle_test]
        nremoved_triangles.put_(indices, torch.ones_like(indices, dtype = torch.long), accumulate = True) # [batchsize]
        
        indices = edge_batch_index[unravel_nomatch_mask]
        nnew_triangles.put_(indices, torch.ones_like(indices, dtype = torch.long), accumulate = True) # [batchsize]
        
        if (nnew_triangles > 2 * nremoved_triangles + 2).sum():
            print(nnew_triangles, nremoved_triangles)
        
        assert (nnew_triangles <= 2 * nremoved_triangles + 2).logical_not().sum().logical_not()
        """
        should always be <=
        can sometimes not be equal
        if equal, this would mean the total removed triangles is determined by the UNIQUE final delaunay triangulation ... and the same would probably be true in any number of dimensions greater than 2 ...
        """
        """
        NOTE:
        I THINK it's possible for nnew_triangles to be less than nremoved_triangles (or my code is just buggy...)
        """
        
        assert nnew_triangles.sum() == batch_newtriangles
        assert nremoved_triangles.sum() == incircle_test.sum()
        
        nadditional_triangles = nnew_triangles - nremoved_triangles # [batchsize]
        ntriangles = self.ntriangles + nadditional_triangles # [batchsize]
        
        partial_delaunay_triangles = torch.empty([ntriangles.sum(), 4], dtype = torch.long, device = device)
        batch_index = torch.empty([ntriangles.sum()], dtype = torch.long, device = device)
        
        cumulative_triangles = torch.cat([torch.zeros([1], dtype = torch.long, device = device), nnew_triangles.cumsum(0)[:-1]]) # [batchsize], cummulative sum starts at zero
        
        """
        since may actually have LESS triangles than previous round, we insert all that survive into the first slots (in that batch)
        """
        good_triangle_indices = torch.arange(incircle_test.logical_not().sum(), dtype = torch.long, device = device)
        good_triangle_indices += cumulative_triangles[self.batch_index[incircle_test.logical_not()]]
        bad_triangle_indices_mask = torch.ones([ntriangles.sum(0)], dtype = torch.bool, device = device)
        bad_triangle_indices_mask.scatter_(0, good_triangle_indices, False)
        
        assert good_triangle_indices.size(0) == incircle_test.logical_not().sum()
        assert bad_triangle_indices_mask.sum() == batch_newtriangles
        
        partial_delaunay_triangles[good_triangle_indices] = self.partial_delaunay_triangles[~incircle_test]
        batch_index[good_triangle_indices] = self.batch_index[~incircle_test]
        
        partial_delaunay_triangles[bad_triangle_indices_mask] = newtriangles
        batch_index[bad_triangle_indices_mask] = nomatch_batch_index
        
        self.partial_delaunay_triangles = partial_delaunay_triangles
        self.batch_index = batch_index
        
        self.ntriangles = ntriangles
        assert list(self.partial_delaunay_triangles.size()) == [self.ntriangles.sum(), 4]
        self.batch_triangles = self.partial_delaunay_triangles.size(0)
        
        self.points_mask.scatter_(
            1, point_index.unsqueeze(1).expand(-1, self.npoints), True
        )
        self.points_sequence = torch.cat(
            [self.points_sequence, point_index.unsqueeze(1)], dim=1
        )
        
        self.cost += nnew_triangles


# turn integer x,y coords (in nxn grid) into position d (0 to n^2-1) along the Hilbert curve.
def xy2d(n, x, y):
    [x, y] = [math.floor(x), math.floor(y)]
    [rx, ry, s, d] = [0, 0, 0, 0]
    s = n / 2
    s = math.floor(s)
    while s > 0:
        rx = (x & s) > 0  # bitwise and, and then boolean is it greater than 0?
        ry = (y & s) > 0
        d += s * s * ((3 * rx) ^ ry)
        [x, y] = rot(n, x, y, rx, ry)
        s = s / 2
        s = math.floor(s)
    return d


def rot(n, x, y, rx, ry):
    if ry == 0:
        if rx == 1:
            x = n - 1 - x
            y = n - 1 - y
        # Swap x and y
        t = x
        x = y
        y = t
    return x, y


def order(
    n, points
):  # turns tensor of points into integer distances along hilbert curve of itteration n
    grid = n * points.to("cpu")
    x = torch.empty([grid.size(0)])
    for i in range(points.size(0)):
        x[i] = xy2d(n, grid[i, 0], grid[i, 1])
    return x


def hilbert_insertion(npoints=103, batchsize=200):
    env.reset(npoints, batchsize)
    points = env.points[:, 3:]  # [batchsize, npoints - 3]
    insertion_order = torch.full([batchsize, npoints], float("inf"), device=device)
    for i in range(batchsize):
        insertion_order[i, 3:] = order(
            2 ** 6, points[i]
        )  # number of possible positions is n ** 2
    for i in range(npoints - 3):
        next_index = insertion_order.min(-1)[1]
        env.update(next_index)
        insertion_order.scatter_(1, next_index.unsqueeze(1), float("inf"))
    print(env.cost.mean().item(), env.cost.var().sqrt().item())


def random_insertion(npoints=104, batchsize=200):
    env.reset(npoints, batchsize)
    for i in range(npoints - 4):
        env.update(torch.full([batchsize], i + 4, dtype=torch.long, device=device))
    print(env.cost.mean().item(), env.cost.var().sqrt().item())

    """
    UNDER CONSTRUCTION
    """
def kdtree_insertion(npoints=103, batchsize=200):
    env.reset(npoints, batchsize)
    points = env.points[:, 3:]  # [batchsize, npoints - 3]

In [281]:
env = environment()
env.reset(54, 10000)
env.update(torch.LongTensor(4 * torch.ones([10000], dtype = torch.long)))
env.update(torch.LongTensor(5 * torch.ones([10000], dtype = torch.long)))
env.update(torch.LongTensor(6 * torch.ones([10000], dtype = torch.long)))
env.update(torch.LongTensor(7 * torch.ones([10000], dtype = torch.long)))
env.update(torch.LongTensor(8 * torch.ones([10000], dtype = torch.long)))
env.update(torch.LongTensor(9 * torch.ones([10000], dtype = torch.long)))
env.update(torch.LongTensor(10 * torch.ones([10000], dtype = torch.long)))
env.update(torch.LongTensor(11 * torch.ones([10000], dtype = torch.long)))
env.update(torch.LongTensor(12 * torch.ones([10000], dtype = torch.long)))
env.update(torch.LongTensor(13 * torch.ones([10000], dtype = torch.long)))
env.update(torch.LongTensor(14 * torch.ones([10000], dtype = torch.long)))
env.update(torch.LongTensor(15 * torch.ones([10000], dtype = torch.long)))
env.update(torch.LongTensor(16 * torch.ones([10000], dtype = torch.long)))
env.update(torch.LongTensor(17 * torch.ones([10000], dtype = torch.long)))
env.update(torch.LongTensor(18 * torch.ones([10000], dtype = torch.long)))
env.update(torch.LongTensor(19 * torch.ones([10000], dtype = torch.long)))
env.update(torch.LongTensor(20 * torch.ones([10000], dtype = torch.long)))
env.update(torch.LongTensor(21 * torch.ones([10000], dtype = torch.long)))
env.update(torch.LongTensor(22 * torch.ones([10000], dtype = torch.long)))
env.update(torch.LongTensor(23 * torch.ones([10000], dtype = torch.long)))
env.update(torch.LongTensor(34 * torch.ones([10000], dtype = torch.long)))
env.update(torch.LongTensor(35 * torch.ones([10000], dtype = torch.long)))
env.update(torch.LongTensor(36 * torch.ones([10000], dtype = torch.long)))
env.update(torch.LongTensor(37 * torch.ones([10000], dtype = torch.long)))
env.update(torch.LongTensor(38 * torch.ones([10000], dtype = torch.long)))
env.update(torch.LongTensor(39 * torch.ones([10000], dtype = torch.long)))
env.update(torch.LongTensor(30 * torch.ones([10000], dtype = torch.long)))
env.update(torch.LongTensor(41 * torch.ones([10000], dtype = torch.long)))
env.update(torch.LongTensor(42 * torch.ones([10000], dtype = torch.long)))
env.update(torch.LongTensor(43 * torch.ones([10000], dtype = torch.long)))
env.update(torch.LongTensor(44 * torch.ones([10000], dtype = torch.long)))
env.update(torch.LongTensor(45 * torch.ones([10000], dtype = torch.long)))
env.update(torch.LongTensor(46 * torch.ones([10000], dtype = torch.long)))
env.update(torch.LongTensor(47 * torch.ones([10000], dtype = torch.long)))
env.update(torch.LongTensor(48 * torch.ones([10000], dtype = torch.long)))
env.update(torch.LongTensor(49 * torch.ones([10000], dtype = torch.long)))
env.update(torch.LongTensor(50 * torch.ones([10000], dtype = torch.long)))
env.update(torch.LongTensor(51 * torch.ones([10000], dtype = torch.long)))
env.update(torch.LongTensor(52 * torch.ones([10000], dtype = torch.long)))
env.update(torch.LongTensor(53 * torch.ones([10000], dtype = torch.long)))

In [21]:
env = environment()
random_insertion(1000, 1000)

KeyboardInterrupt: 

In [2]:
env = environment()
random_insertion(100, 1000)

1804.4581298828125 49.614105224609375


In [244]:
env.partial_delaunay_triangles[env.batch_index == 11]

tensor([[5, 4, 0, 1],
        [5, 0, 2, 1],
        [4, 0, 1, 3],
        [5, 4, 3, 0],
        [5, 0, 3, 2],
        [5, 4, 1, 3],
        [5, 1, 2, 3]])

In [122]:
env.points[4]

tensor([[0.0000, 0.0000, 0.0000],
        [3.0000, 0.0000, 0.0000],
        [0.0000, 3.0000, 0.0000],
        [0.0000, 0.0000, 3.0000],
        [0.8572, 0.7212, 0.6252],
        [0.5582, 0.0759, 0.8407],
        [0.8818, 0.4546, 0.6969]], dtype=torch.float64)

In [85]:
env.cost[30]

tensor(6., dtype=torch.float64)

In [69]:
env.batch_index

tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  2,  2,  2,
         2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,  4,  4,  4,  4,
         5,  5,  5,  5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  6,  6,  7,  7,  7,
         7,  7,  7,  7,  8,  8,  8,  8,  8,  8,  8,  9,  9,  9,  9,  9,  9,  9,
        10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12,
        12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 15,
        15, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17,
        17, 17, 18, 18, 18, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 19, 19, 20,
        20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 21, 22, 22, 22, 22, 22, 22,
        23, 23, 23, 23, 23, 23, 23, 24, 24, 24, 24, 24, 24, 24, 24, 25, 25, 25,
        25, 25, 25, 25, 25, 26, 26, 26, 26, 26, 26, 26, 27, 27, 27, 27, 27, 27,
        27, 28, 28, 28, 28, 28, 28, 28, 28, 29, 29, 29, 29, 29, 29, 29, 29, 30,
        30, 30, 30, 30, 30, 30, 31, 31, 

In [50]:
env.batch_index

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

tensor([[6, 4, 2, 0],
        [5, 4, 3, 0],
        [4, 0, 1, 3],
        [5, 4, 2, 3],
        [6, 4, 1, 2],
        [5, 0, 3, 2],
        [6, 4, 0, 1],
        [6, 0, 2, 1],
        [5, 4, 1, 3],
        [5, 1, 2, 3],
        [6, 5, 0, 4],
        [6, 5, 2, 0],
        [6, 5, 4, 2],
        [6, 4, 0, 2],
        [6, 5, 2, 4],
        [6, 5, 1, 2],
        [6, 5, 4, 1],
        [6, 4, 2, 1],
        [6, 5, 3, 4],
        [6, 5, 2, 3],
        [6, 5, 4, 2],
        [6, 4, 3, 2]])

In [52]:
env.cost

tensor([32., 36.], dtype=torch.float64)

In [46]:
env.batch_index

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [3]:
a = torch.BoolTensor([1, 0, 1])

In [4]:
a

tensor([ True, False,  True])

In [7]:
a.sum().bool()

tensor(True)

In [41]:
d

tensor([[False, False, False, False],
        [ True,  True,  True,  True],
        [ True,  True,  True,  True],
        [ True,  True,  True,  True]])

In [2]:
a=[]
for i in range(1000):
    a.append(torch.rand([10,10], device = "cuda"))

In [46]:
a

[tensor([[0.4313, 0.9969, 0.5534, 0.4680, 0.4442, 0.4795, 0.5959, 0.2111, 0.1494,
          0.2935],
         [0.2582, 0.1583, 0.2474, 0.7711, 0.7818, 0.0457, 0.8337, 0.2097, 0.9068,
          0.2732],
         [0.4875, 0.5738, 0.3785, 0.3103, 0.4578, 0.6669, 0.5239, 0.9867, 0.3946,
          0.0521],
         [0.2193, 0.9995, 0.1599, 0.7146, 0.5984, 0.0783, 0.2196, 0.4543, 0.2215,
          0.5127],
         [0.8459, 0.5896, 0.2575, 0.9742, 0.6783, 0.6695, 0.3953, 0.1152, 0.9243,
          0.4190],
         [0.5693, 0.3542, 0.3679, 0.7874, 0.7078, 0.3503, 0.3533, 0.5786, 0.4232,
          0.7289],
         [0.3985, 0.2199, 0.5201, 0.8040, 0.1900, 0.7452, 0.9780, 0.2237, 0.5022,
          0.6029],
         [0.2554, 0.0275, 0.5087, 0.4870, 0.3547, 0.7727, 0.6715, 0.2615, 0.0614,
          0.0894],
         [0.6333, 0.8916, 0.4596, 0.2480, 0.7392, 0.8683, 0.6115, 0.5775, 0.7711,
          0.7761],
         [0.0666, 0.9018, 0.0967, 0.8879, 0.6802, 0.3070, 0.9359, 0.2990, 0.4182,
         

In [51]:
%timeit torch.cat(a)
b=torch.cat(a)

785 µs ± 13.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [53]:
%timeit b.to("cpu")

135 µs ± 1.35 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [56]:
c = b.to("cpu")
%timeit c.to("cuda")

115 µs ± 1.01 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [57]:
c.size()

torch.Size([10000, 10])

In [3]:
%%timeit
for i in range(len(a)):
    torch.cat([a[i], a[i]])

19 ms ± 211 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


tensor([[0.5375, 0.4782, 0.0017, 0.1198, 0.4668, 0.6339, 0.9158, 0.4345, 0.0299,
         0.0123],
        [0.9989, 0.3435, 0.6925, 0.7289, 0.8454, 0.1467, 0.6666, 0.3206, 0.8522,
         0.5354],
        [0.2968, 0.9030, 0.9967, 0.6800, 0.1578, 0.1072, 0.7633, 0.9154, 0.8514,
         0.1419],
        [0.0462, 0.3935, 0.1409, 0.0580, 0.9880, 0.8230, 0.9829, 0.0964, 0.8038,
         0.0132],
        [0.6265, 0.3549, 0.8174, 0.4463, 0.0033, 0.6377, 0.2873, 0.2666, 0.4382,
         0.9503],
        [0.7307, 0.4144, 0.3998, 0.2687, 0.9614, 0.0297, 0.9542, 0.6001, 0.2971,
         0.8017],
        [0.7009, 0.9257, 0.0758, 0.8964, 0.5289, 0.5252, 0.0342, 0.9832, 0.2709,
         0.4401],
        [0.5478, 0.8381, 0.6545, 0.7272, 0.8688, 0.2488, 0.8865, 0.2477, 0.6909,
         0.9043],
        [0.2142, 0.9939, 0.8075, 0.2584, 0.1045, 0.9718, 0.2924, 0.7600, 0.0915,
         0.8136],
        [0.3359, 0.1303, 0.2264, 0.9342, 0.9507, 0.5101, 0.7184, 0.2601, 0.5989,
         0.6838]], device='c

In [6]:
torch.zeros([1]).size()

torch.Size([1])

In [8]:
a = torch.rand([4,5])
a

tensor([[0.8953, 0.8995, 0.2129, 0.4331, 0.6116],
        [0.8755, 0.5284, 0.1661, 0.1650, 0.0834],
        [0.7336, 0.7470, 0.9222, 0.6837, 0.1013],
        [0.5622, 0.6021, 0.7589, 0.1455, 0.9529]])

In [100]:
b = torch.BoolTensor([[False, True], [True, False]])
b == ~b

tensor([[False, False],
        [False, False]])

In [102]:
c = torch.BoolTensor([[True, True], [False, False]])

In [103]:
b == c

tensor([[False,  True],
        [False,  True]])

In [109]:
a=torch.randint(2, [4,4])
(a == -a).sum(-1), a

(tensor([2, 3, 2, 3]),
 tensor([[1, 0, 1, 0],
         [0, 0, 1, 0],
         [1, 1, 0, 0],
         [0, 1, 0, 0]]))

In [31]:
a.put_(b.nonzero(as_tuple = True)[0], 10 * torch.ones([2], dtype = torch.float))

tensor([[ 0.3098, 10.0000, 10.0000,  0.9444],
        [ 0.7635,  0.7679,  0.6431,  0.5433],
        [ 0.0805,  0.9476,  0.3478,  0.6309],
        [ 0.0301,  0.8120,  0.9441,  0.4770]])

In [91]:
a = torch.tensor([1, -1]).bool()

In [92]:
a.sum()

tensor(2)